## Dump the model to Kipoi

In [45]:
import numpy as np
import os
import m_kipoi
from copy import deepcopy
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
from m_kipoi.utils import read_pkl
from m_kipoi.config import get_data_dir
from tqdm import tqdm
ddir = get_data_dir()

### Data

#### ClinVar

In [3]:
clinvar_file = f"{ddir}/processed/splicing/clinvar/annotated_vcf/20180429.filtered/clinvar_ext_Xy.pkl"
X_clinvar, y_clinvar = read_pkl(clinvar_file)

In [4]:
X_clinvar['early_stop'] = X_clinvar.early_stop.astype(bool)

In [5]:
y_clinvar = y_clinvar[~X_clinvar['early_stop']]
X_clinvar = X_clinvar[~X_clinvar['early_stop']]

## Features

In [2]:
# Use these Kipoi models
models = ["MaxEntScan/3prime", "MaxEntScan/5prime", "HAL", "labranchor"]

In [25]:
kipoi_features = ['MaxEntScan/3prime_alt',
                  'MaxEntScan/3prime_ref',
                  'MaxEntScan/3prime_isna',
                  'MaxEntScan/5prime_alt',
                  'MaxEntScan/5prime_ref',
                  'MaxEntScan/5prime_isna',
                  'HAL_ref',
                  'HAL_alt',
                  'HAL_isna',
                  'labranchor_logit_alt',
                  'labranchor_logit_ref',
                  'labranchor_isna']

In [26]:
conservation_features = ['phyloP46way_placental', 'phyloP46way_primate', 'CADD_raw', 'CADD_phred']

In [35]:
clinvar_kipoi_features = kipoi_features + conservation_features

## Modeling

In [36]:
# Scikit-learn imports
import sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn_pandas import DataFrameMapper
from sklearn.externals import joblib

In [37]:
class ZeroImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return pd.DataFrame(X).fillna(0).copy()

def preproc(features):
    """Pre-processing pipeline
    """
    return DataFrameMapper([
            (features, [ZeroImputer()]),
        ])

In [52]:
X = preproc(kipoi_features).fit_transform(X_clinvar)
model = Pipeline([('preproc', sklearn.preprocessing.StandardScaler()),
                  ('model', LogisticRegression())])
model.fit(X, y_clinvar)
mdir = os.path.expanduser('~/.kipoi/models/KipoiSplice/4')
os.makedirs(os.path.join(mdir, "model_files"), exist_ok=True)
joblib.dump(model, f'{mdir}/model_files/model.pkl')

['/data/ouga/home/ag_gagneur/avsec/.kipoi/models/KipoiSplice/4/model_files/model.pkl']

In [53]:
X = preproc(clinvar_kipoi_features).fit_transform(X_clinvar)
model = Pipeline([('preproc', sklearn.preprocessing.StandardScaler()),
                  ('model', LogisticRegression())])
model.fit(X, y_clinvar)
mdir = os.path.expanduser('~/.kipoi/models/KipoiSplice/4cons')
os.makedirs(os.path.join(mdir, "model_files"), exist_ok=True)
joblib.dump(model, f'{mdir}/model_files/model.pkl')

['/data/ouga/home/ag_gagneur/avsec/.kipoi/models/KipoiSplice/4cons/model_files/model.pkl']

In [133]:
def evaluate(df, y, features, model, model_name):
        ret = cross_validate(Pipeline([('preproc', preproc(features)), 
                                       ('model', model)]), 
                             df, y, scoring=['accuracy', 'roc_auc'], cv=10, n_jobs=10, return_train_score=True)
        means = pd.DataFrame(ret).describe().loc['mean']
        means.index = "mean_" + means.index
        sds = pd.DataFrame(ret).describe().loc['std']
        sds.index = "std_" + sds.index
        return pd.DataFrame([{**dict(means), **dict(sds), "model_name": model_name}])

In [134]:
def run_model_groups(df, y, model_groups,
                     model=LogisticRegressionCV(penalty="l1", solver='liblinear', scoring="roc_auc", cv=3, n_jobs=1)):
    res = []
    use_features = []
    for mg in tqdm(model_groups):
        use_features += [f for f in df.columns if f.startswith(mg)]
        res.append(evaluate(df, y, use_features, model, model_name=mg))
    return pd.concat(res)

In [316]:
res_dbscsnv = run_model_groups(X_dbscsnv, y_dbscsnv, models, model=model)

100%|██████████| 4/4 [00:05<00:00,  1.44s/it]


In [317]:
res_dbscsnv = res_dbscsnv.append(evaluate(X_dbscsnv, y_dbscsnv, kipoi_features + conservation_features, 
                                          model=model,
                                          model_name="Kipoi4 w/ cons."))